In [30]:
import grammar as g
import tmp as t
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import importlib



In [4]:
gramm = g.Grammar('/home/alexserra98/uni/prog_deep/Genetics_ANN/GA_for_param_optim/cnn.grammar.txt')


In [5]:
for i in gramm.grammar:
    print(f'{i} : {gramm.grammar[i]}')

features : [[('convolution', True), ('pooling', True)], [('convolution', True)], [('pooling', True)], [('pooling', True)], [('dropout', True)], [('batch-norm', True)]]
convolution : [[('layer:conv', False), ('[num-filters,int,1,32,256]', False), ('[filter-shape,int,1,2,5]', False), ('[stride,int,1,1,3]', False), ('padding', True), ('activation-function', True), ('bias', True)]]
batch-norm : [[('layer:batch-norm', False)]]
pooling : [[('pool-type', True), ('[kernel-size,int,1,2,5]', False), ('[stride,int,1,1,3]', False), ('padding', True)]]
pool-type : [[('layer:pool-avg', False)], [('layer:pool-max', False)]]
padding : [[('padding:same', False)], [('padding:valid', False)]]
dropout : [[('layer:dropout', False), ('[rate,float,1,0,0.7]', False)]]
classification : [[('fully-connected', True)], [('dropout', True)]]
fully-connected : [[('layer:fc', False), ('activation-function', True), ('[num-units,int,1,128,2048]', False), ('bias', True)]]
activation-function : [[('act:linear', False)], [

In [6]:
gramm.grammar['convolution'][0]

[('layer:conv', False),
 ('[num-filters,int,1,32,256]', False),
 ('[filter-shape,int,1,2,5]', False),
 ('[stride,int,1,1,3]', False),
 ('padding', True),
 ('activation-function', True),
 ('bias', True)]

In [16]:
prova = gramm.initialise('features')

In [17]:
for i in prova:
    print(f'{i} : {prova[i]}')

features : [{'ge': 0, 'ga': {}}]
convolution : [{'ge': 0, 'ga': {'num-filters': ('int', 32.0, 256.0, [125]), 'filter-shape': ('int', 2.0, 5.0, [5]), 'stride': ('int', 1.0, 3.0, [3])}}]
padding : [{'ge': 1, 'ga': {}}, {'ge': 0, 'ga': {}}]
activation-function : [{'ge': 1, 'ga': {}}]
bias : [{'ge': 1, 'ga': {}}]
pooling : [{'ge': 0, 'ga': {'kernel-size': ('int', 2.0, 5.0, [4]), 'stride': ('int', 1.0, 3.0, [1])}}]
pool-type : [{'ge': 0, 'ga': {}}]


In [18]:
type(prova['features'])

list

In [19]:
feat_layers = gramm.decode('features', prova)

In [20]:
layer_decoding = t.Evaluator()
l_dict = layer_decoding.get_layers(feat_layers) 
l_dict

[('conv',
  {'num-filters': ['125'],
   'filter-shape': ['5'],
   'stride': ['3'],
   'padding': ['valid'],
   'act': ['relu'],
   'bias': ['False']}),
 ('pool-avg', {'kernel-size': ['4'], 'stride': ['1'], 'padding': ['same']})]

In [21]:
def get_act(name):
    if name == 'linear':
        return nn.Linear()
    elif name == 'relu':
        return nn.ReLU()
    elif name == 'sigmoid':
        return nn.Sigmoid()

get_act(l_dict[0][1]['act'][0])

ReLU()

In [22]:
l_dict[0][1]['act'][0]

'relu'

In [61]:
type(l_dict)

list

In [80]:
importlib.reload(t)

<module 'tmp' from '/home/alexserra98/uni/prog_deep/Genetics_ANN/GA_for_param_optim/tmp.py'>

In [81]:

assembler = t.Evaluator()
lay = assembler.assemble_network(l_dict,(28,28,3))

layer_type: conv
layer_type: pool-avg
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 125, 8, 8]           9,375
              ReLU-2            [-1, 125, 8, 8]               0
         AvgPool2d-3            [-1, 125, 5, 5]               0
Total params: 9,375
Trainable params: 9,375
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.15
Params size (MB): 0.04
Estimated Total Size (MB): 0.19
----------------------------------------------------------------


In [67]:
type(lay)

NameError: name 'lay' is not defined

In [44]:
def compute_output_conv2d(input_shape,out_channel, kernel_size, stride, padding, dilation=1):
    """
        Compute the output shape after a conv layer.
        Parameters
        ----------
        input_shape : tuple
            (width, length, channel)
        out_channel : int
        kernel_size : int
        stride: int
        padding : int
        dilation : int
        
        Return
        ----------
        output : tuple
            (width, length, channel)

    """
    if isinstance(input_shape, tuple) and len(input_shape)==3:
        return (int((input_shape[0] + 2*padding[0] - dilation[0]*(kernel_size[0]-1) - 1)/stride[0] + 1),
            int((input_shape[1] + 2*padding[1] - dilation[1]*(kernel_size[1]-1) - 1)/stride[1] + 1), out_channel)
    elif isinstance(input_shape, tuple) and len(input_shape)==2:
        return (int((input_shape[0] + 2*padding[0] - dilation[0]*(kernel_size[0]-1) - 1)/stride[0] + 1),
            int((input_shape[1] + 2*padding[1] - dilation[1]*(kernel_size[1]-1) - 1)/stride[1] + 1))
    else:
        return int((input_shape + 2*padding - dilation*(kernel_size-1) - 1)/stride + 1)

compute_output_conv2d((28,28,3),1,[1,1],[2,2],[3,3],[1,1])

(17, 17, 1)